In [ ]:
# Install packages if not present

#pip install coiled==0.2.13
#pip install dask==2022.03.0
#pip install itk==5.3rc4.post2
#pip install vtk
#pip install git+https://github.com/uncbiag/OAI_analysis_2.git

In [41]:
# All Imports

import coiled
import dask
from dask import compute, visualize, delayed

import time
import random
import itk
import vtk
import oai_analysis_2
from oai_analysis_2 import dask_processing as dp

In [81]:
# Start Coiled Cluster
# Enter Token when asked

import coiled
import dask
from dask import compute, visualize, delayed

#dask.config.set({'distributed.scheduler.worker-ttl' : '5m'})

with dask.config.set({"dask.distributed.scheduler.worker-ttl": "5m", "distributed.worker.resources.GPU": 1}):
    cluster = coiled.Cluster(
        name='gpucluster35',
        scheduler_vm_types=['t3.medium'],
        worker_vm_types=["g4dn.2xlarge", "g5.2xlarge", "p3.2xlarge", "p2.xlarge"],
        n_workers=6,
        software="pranjal-sahu/oaiimageanalysis",
        shutdown_on_close=True,
        backend_options={"multizone":True}
    )

client = dask.distributed.Client(cluster)

Output()

Mismatched versions found

+-------------+-----------+-----------+----------+
| Package     | client    | scheduler | workers  |
+-------------+-----------+-----------+----------+
| blosc       | 1.10.6    | MISSING   | MISSING  |
| cloudpickle | 2.0.0     | 2.1.0     | 2.1.0    |
| dask        | 2022.03.0 | 2022.6.0  | 2022.6.0 |
| distributed | 2022.3.0  | 2022.6.0  | 2022.6.0 |
| numpy       | 1.21.5    | 1.21.6    | 1.21.6   |
| pandas      | 1.3.1     | 1.4.2     | 1.4.2    |
+-------------+-----------+-----------+----------+


In [46]:
@delayed
def download():
    import os
    import hashlib
    import urllib
    
    for i in range(5):
        if os.path.exists('pretrained_OAI_model'):        
            h = hashlib.md5(open('pretrained_OAI_model', 'rb').read()).hexdigest()
            if h == '414e4b5da9f62a4ab3635791b52cbdfc':
                return h
        else:
            urllib.request.urlretrieve(
                "https://github.com/HastingsGreer/InverseConsistency/releases/download/pretrained_oai_model/knee_aligner_resi_net99900",
        "pretrained_OAI_model")
    return 1

In [47]:
for i in range(20):
    joblist = []
    temp = download()
    joblist.append(temp)
    compute(*joblist)

In [ ]:
# @delayed
# def clearcache():
#     import gc
#     from numba import cuda
#     cuda.select_device(0)
#     cuda.close()
#     gc.collect()
#     return

# joblist = []

# for i in range(100):
#     temp = clearcache()
#     joblist.append(temp)
# compute(*joblist)

In [60]:
def inc(x):
    time.sleep(random.random())
    return x + 1

inc(1)

2

In [295]:
patients_array = [
    '9283903', '9094865', '9290211', '9171580', '9214480', '9208055',
    '9000798', '9138965', '9246995', '9291234', '9068305', '9101854',
    '9316405', '9305065', '9075815', '9036316', '9155449', '9247140',
    '9128011', '9297051', '9285212', '9054866', '9007827', '9261513',
    '9263504', '9037823', '9021791', '9210505', '9239017', '9268652', 
    '9135902', '9252629', '9028786', '9277154', '9103365', 
    '9016304', '9233675', '9235073', '9212530', '9040390'
]

months_array = [
    'Month-0', 'Month-12', 'Month-24', 'Month-36', 'Month-48', 'Month-72',
    'Month-96'
]


i = 0
batch_size = 1
patient_result = []
start_time = time.time()
patients_array_temp = patients_array[i:min(i + batch_size, len(patients_array))]

for patient_id in patients_array_temp:
    print(patient_id, time.time())
    for month in patients_array_months_dict[patient_id]:
        image_path = "s3://oaisample1/ZARRDATA/PatientID-" + patient_id + '/' + month + '/Images/SAG_3D_DESS_1.zarr/scale0'
        fc_probmap, tc_probmap = my_segment_method(image_path)
        patient_result.append(fc_probmap)
    
print(patient_result)

9283903 1669261549.2987666
[Delayed('getitem-14439b9dea2ae6bec05e607b39a34757'), Delayed('getitem-f7d8b1adab14063ecf8fcb5bb341f9d1'), Delayed('getitem-62b0330263d7e118bd8f9fb8fc269562'), Delayed('getitem-2af03f6f48b18e8b1fa261da661111cd'), Delayed('getitem-19c9de44d507fcae060ba472dcb19d39'), Delayed('getitem-4bb90e575127dc27298c42d7b8389ceb'), Delayed('getitem-1ec4ab9763ffa7d44bded03dc5ca93b6')]


In [68]:
# Main execution loop
# Change the patients_array and months_array based on requirements.
# Following code will run the computation on all samples.

atlas_path = "s3://oaisample1/ZARRDATA/atlas_image.zarr/scale0"
write_path = "/data/OAIFULLDATA/RESULTS2/"

patients_array = [
    '9283903', '9094865', '9290211', '9171580', '9214480', '9208055',
    '9000798', '9138965', '9246995', '9291234', '9068305', '9101854',
    '9316405', '9305065', '9075815', '9036316', '9155449', '9247140',
    '9128011', '9297051', '9285212', '9054866', '9007827', '9261513',
    '9263504', '9037823', '9021791', '9210505', '9239017', '9268652', 
    '9135902', '9252629', '9028786', '9277154', '9103365', 
    '9016304', '9233675', '9235073', '9212530', '9040390'
]

months_array = [
    'Month-0', 'Month-12', 'Month-24', 'Month-36', 'Month-48', 'Month-72',
    'Month-96'
]

counter = 0

import json
f = open('patients_array_months_dict_right.json',)
patients_array_months_dict = json.load(f)

def image_normalize(image, window_min_perc, window_max_perc, output_min, output_max):
    import itk
    import numpy as np
    from itk import IntensityWindowingImageFilter as IntensityWindowingImageFilter

    window_rescale = IntensityWindowingImageFilter[type(image), type(image)].New()
    image_array = itk.GetArrayFromImage(image)

    window_min = np.percentile(image_array, window_min_perc)
    window_max = np.percentile(image_array, window_max_perc)

    window_rescale.SetInput(image)
    window_rescale.SetOutputMaximum(output_max)
    window_rescale.SetOutputMinimum(output_min)
    window_rescale.SetWindowMaximum(window_max)
    window_rescale.SetWindowMinimum(window_min)
    window_rescale.Update()
    return window_rescale.GetOutput()

def readimage(image_path):
    import itk
    from itk import CastImageFilter as CastImageFilter
    import xarray as xr

    xr_dataset = xr.open_zarr(image_path)
    xr_array = xr_dataset.get("image")
    itk_image = itk.image_from_xarray(xr_array)

    cast_filter_type = CastImageFilter[type(itk_image), itk.Image[itk.F, 3]].New()
    cast_filter_type.SetInPlace(False)
    cast_filter_type.SetInput(itk_image)
    cast_filter_type.Update()
    itk_image = cast_filter_type.GetOutput()
    return itk_image

#@delayed(nout=2)
def my_segment_method(image_A):
    import itk
    from itk import IntensityWindowingImageFilter as IntensityWindowingImageFilter
    import oai_analysis_2
    import torch
    import os
    from os.path import exists
    from oai_analysis_2 import utils
    from oai_analysis_2.segmentation import segmenter
    import numpy as np
    import urllib.request
    import gc

    if torch.cuda.is_available():
        device = "cuda"
        torch.cuda.empty_cache()
    else:
        print("WARNING: CUDA NOT AVAILABLE, FALLING BACK TO CPU")
        device = "cpu"

    if not exists("segmentation_model.pth.tar"):
        print("Downloading segmentation model")
        urllib.request.urlretrieve(
            "https://github.com/uncbiag/OAI_analysis_2/blob/master/data/segmentation_model.pth.tar?raw=true",
            "segmentation_model.pth.tar",
        )

    if not exists("segmentation_train_config.pth.tar"):
        urllib.request.urlretrieve(
            "https://github.com/uncbiag/OAI_analysis_2/blob/master/data/segmentation_train_config.pth.tar?raw=true",
            "segmentation_train_config.pth.tar",
        )

    # Initialize segmenter
    segmenter_config = dict(
        ckpoint_path="segmentation_model.pth.tar",
        training_config_file="segmentation_train_config.pth.tar",
        device=device,
        batch_size=2,
        overlap_size=(16, 16, 8),
        output_prob=True,
        output_itk=True,
    )

    segmenter = oai_analysis_2.segmentation.segmenter.Segmenter3DInPatchClassWise(
        mode="pred", config=segmenter_config
    )

    # Segment downloaded image
    test_volume = readimage(image_A)

    test_volume = image_normalize(test_volume, 0.1, 99.9, 0, 1)

    FC_probmap, TC_probmap = segmenter.segment(
        test_volume, if_output_prob_map=True, if_output_itk=True
    )

    del segmenter
    gc.collect()

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return FC_probmap, TC_probmap

#@delayed(nout=2)
def deform_and_get_thickness(phi_AB, image_A, image_B, prob, image_type="FC"):
    import itk
    import numpy as np
    from oai_analysis_2 import mesh_processing as mp
    from itk import LinearInterpolateImageFunction

    interpolator = LinearInterpolateImageFunction.New(image_A)
    warped_image = itk.resample_image_filter(
        prob,
        transform=phi_AB,
        interpolator=interpolator,
        size=itk.size(image_B),
        output_spacing=itk.spacing(image_B),
        output_direction=image_B.GetDirection(),
        output_origin=image_B.GetOrigin(),
    )
    
    mesh_type = image_type
    distance_inner, _ = mp.get_thickness_mesh(warped_image, mesh_type=mesh_type)
    distance_inner_itk = mp.get_itk_mesh(distance_inner)
    return warped_image, distance_inner_itk

#@delayed(nout=3)
def register_images_delayed_gpu(image_A, image_B):
    import itk
    from itk import Image as itkImage
    from itk import CastImageFilter as CastImageFilter
    import icon_registration
    import icon_registration.itk_wrapper as itk_wrapper
    import icon_registration.pretrained_models as pretrained_models
    import torch
    import xarray as xr
    import time
    import numpy as np
    import gc

    image_A = readimage(image_A)
    image_B = readimage(image_B)

    cast_filter_type = CastImageFilter[type(image_A), itkImage[itk.D, 3]].New()
    cast_filter_type.SetInPlace(False)
    cast_filter_type.SetInput(image_A)
    cast_filter_type.Update()
    image_A = cast_filter_type.GetOutput()

    cast_filter_type = CastImageFilter[type(image_B), itkImage[itk.D, 3]].New()
    cast_filter_type.SetInPlace(False)
    cast_filter_type.SetInput(image_B)
    cast_filter_type.Update()
    image_B = cast_filter_type.GetOutput()

    image_A = image_normalize(image_A, 0.1, 99.9, 0, 1)

    model = pretrained_models.OAI_knees_gradICON_model()
    model.cuda()

    # Register the images
    phi_AB, phi_BA = itk_wrapper.register_pair(model, image_A, image_B)

    del model
    gc.collect()
    return phi_AB, image_A, image_B


import time
print('Start Time ', time.time())

# for patient_id_a in range(0, len(patients_array), 3):
#     patient_id1 = patient_id_a
#     patient_id2 = patient_id_a + 1
#     patient_id3 = patient_id_a + 2
    
#     print(patients_array[patient_id1], time.time())
          
#     patient_ids = [patients_array[patient_id1], patients_array[patient_id2], patients_array[patient_id3]]
    
#     patient_result = []
#     segmentation_result = []

#     for patient_id in patient_ids:
#         for month in patients_array_months_dict[patient_id]:
#             image_path = "s3://oaisample1/ZARRDATA/PatientID-" + patient_id + '/' + month + '/Images/SAG_3D_DESS_1.zarr/scale0'

#             fc_probmap, tc_probmap = dp.segment_method(image_path)
#             phi_AB, image_A, image_B = dp.register_images_delayed(image_path, atlas_path)

#             deformed_fc, thickness_fc = deform_and_get_thickness(phi_AB, image_A, image_B,
#                                                  fc_probmap, 'FC')
#             deformed_tc, thickness_tc = deform_and_get_thickness(phi_AB, image_A, image_B,
#                                                  tc_probmap, 'TC')
#             #thickness_fc = dp.get_thickness(deformed_fc, 'FC')
#             #thickness_tc = dp.get_thickness(deformed_tc, 'TC')
#             #patient_result.append([deformed_fc, deformed_tc, thickness_fc, thickness_tc])
#             patient_result.append([ thickness_fc, thickness_tc])

#    output_result = compute(*patient_result)

#with dask.annotate(resources={'GPU': 1}):

# import numpy as np

# batch_size = 2
# for i in np.arange(0, len(patients_array), batch_size):
#     patient_result = []
#     start_time = time.time()
#     patients_array_temp = patients_array[i:min(i + batch_size, len(patients_array))]
#     for patient_id in patients_array_temp:
#         print(patient_id, time.time())
#         for month in patients_array_months_dict[patient_id]:
#             image_path = "s3://oaisample1/ZARRDATA/PatientID-" + patient_id + '/' + month + '/Images/SAG_3D_DESS_1.zarr/scale0'

#             fc_probmap, tc_probmap = my_segment_method(image_path)
#             phi_AB, image_A, image_B = dp.register_images_delayed(image_path, atlas_path)

#             deformed_fc = dp.deform_probmap_delayed(phi_AB, image_A, image_B,
#                                                  fc_probmap, 'FC')
#             deformed_tc = dp.deform_probmap_delayed(phi_AB, image_A, image_B,
#                                                  tc_probmap, 'TC')
#             thickness_fc = dp.get_thickness(deformed_fc, 'FC')
#             thickness_tc = dp.get_thickness(deformed_tc, 'TC')

#             #thickness_fc = dp.get_thickness(deformed_fc, 'FC')
#             #thickness_tc = dp.get_thickness(deformed_tc, 'TC')
#             #patient_result.append([deformed_fc, deformed_tc, thickness_fc, thickness_tc])
#             patient_result.append([ thickness_fc, thickness_tc])
#         #print('Total tasks ', len(patient_result))
#     output_result = compute(*patient_result)
#     end_time = time.time()
#     print("Total time ", end_time - start_time)
    
#     for idx, result in enumerate(output_result):
#         itk.imwrite(
#             result[0], write_path + patient_id + '_' +
#             str(counter) + '_itk_deformed_FC.nii.gz')
#         itk.imwrite(
#             result[1], write_path + patient_id + '_' +
#             str(counter) + '_itk_deformed_TC.nii.gz')

#         itk.meshwrite(
#             result[2], write_path + patient_id + '_' +
#             str(counter) + '_itk_distance_inner_FC.vtk')
#         itk.meshwrite(
#             result[3], write_path + patient_id + '_' +
#             str(counter) + '_itk_distance_inner_TC.vtk')

#         counter = counter + 1

print(time.time())

Start Time  1669313091.327517
1669313091.3276026


In [61]:
future = client.submit(inc, 1)
future

<Future: pending, key: inc-cc1d2d0e9d34dc8b24bf18499f6f23ec>

In [234]:
# 13 workers, single thread, 5 patients
# Start Time  1669226166.696405
# 9283903 1669226166.6965768
# 9094865 1669226166.6982548
# 9290211 1669226166.7007987
# 9171580 1669226166.7022889
# 9214480 1669226166.7045743
# Total time  577.3136601448059
# 9208055 1669226744.0105462
# 9000798 1669226744.0123682
# 9138965 1669226744.0141132
# 9246995 1669226744.015336
# 9291234 1669226744.017079
# Total time  637.0120487213135
# 9068305 1669227381.022912
# 9101854 1669227381.0252223
# 9316405 1669227381.0264766
# 9305065 1669227381.0309145
# 9075815 1669227381.0323849
# Total time  589.5682468414307
# 9036316 1669227970.591917
# 9155449 1669227970.5938416
# 9247140 1669227970.5955162
# 9128011 1669227970.5972428
# 9297051 1669227970.5986245
# Total time  471.98714900016785
# 9285212 1669228442.5795484
# 9054866 1669228442.5803204
# 9007827 1669228442.5817335
# 9261513 1669228442.5830693
# 9263504 1669228442.584496
# Total time  332.5523314476013
# 9037823 1669228775.1323252
# 9021791 1669228775.133828
# 9210505 1669228775.1358924
# 9239017 1669228775.1372588
# 9268652 1669228775.1386793
# Total time  614.2152674198151
# 9135902 1669229389.3479235
# 9252629 1669229389.3494983
# 9028786 1669229389.351675
# 9277154 1669229389.3530786
# 9103365 1669229389.354506
# Total time  524.350536108017
# 9016304 1669229913.698951
# 9233675 1669229913.7006352
# 9235073 1669229913.7019432
# 9212530 1669229913.7032893
# 9040390 1669229913.7047756
# Total time  608.9414417743683
# 1669230522.6409862

array([ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80,
       85, 90, 95])

In [236]:
cluster.scale(8)

In [50]:

patients_array = [
    '9283903', '9094865', '9290211', '9171580', '9214480', '9208055',
    '9000798', '9138965', '9246995', '9291234', '9068305', '9101854',
    '9316405', '9305065', '9075815', '9036316', '9155449', '9247140',
    '9128011', '9297051', '9285212', '9054866', '9007827', '9261513',
    '9263504', '9037823', '9021791', '9210505', '9239017', '9268652', 
    '9135902', '9252629', '9028786', '9277154', '9103365', 
    '9016304', '9233675', '9235073', '9212530', '9040390'
]

months_array = [
    'Month-0', 'Month-12', 'Month-24', 'Month-36', 'Month-48', 'Month-72',
    'Month-96'
]


patient_id = patients_array[0]
image_paths = []
for month in patients_array_months_dict[patient_id]:
    image_path = "s3://oaisample1/ZARRDATA/PatientID-" + patient_id + '/' + month + '/Images/SAG_3D_DESS_1.zarr/scale0'
    image_paths.append(image_path)

#print(patient_result)
#fc_probmap, tc_probmap = my_segment_method(image_path)
#patient_result.append([fc_probmap, tc_probmap])

#segmentation_masks = [client.submit(my_segment_method, d, resources={'GPU': 1}) for d in patient_result]
#registered_images = [client.submit(register_images_delayed_gpu, (d, atlas_path), resources={'GPU': 1}) for d in patient_result]


#print(processed)

In [53]:
client

<Client: 'tls://10.0.61.181:8786' processes=8 threads=64, memory=240.91 GiB>

In [75]:
# MAIN EXECUTION USING Futures

segmentation_masks = []
registered_images = []

for i in range(len(image_paths)):
    segmentation_mask = client.submit(my_segment_method, image_paths[i], resources={'GPU': 1})
    segmentation_masks.append(segmentation_mask)
    
    registered_image = client.submit(register_images_delayed_gpu, image_paths[i], atlas_path, resources={'GPU': 1})
    registered_images.append(registered_image)
    
    #thickness_fc = client.submit(deform_and_get_thickness, (registered_image[0], registered_image[1], registered_image[2], segmentation_mask[0], 'FC'))
    #thickness_tc = client.submit(deform_and_get_thickness, (registered_image[0], registered_image[1], registered_image[2], segmentation_mask[1], 'TC'))


In [78]:
for i in range(len(registered_images)):
    print(segmentation_masks[i].release())

None
None
None
None
None
None
None


In [ ]:
futures = []

for i in range(3):
    future = client.submit(my_segment_method, patient_result[i])
    futures.append(future)

result = [future.result() for future in futures]
result

In [118]:
#cluster.scale(10)

# Average per patient
# 105 seconds with 6 or 10 (have to confirm) GPUs
# 173 seconds with 8 GPUs (5 patients)
# x seconds with 8 GPUs (10 patients)

In [95]:
# Cluster 10, threads 1, 2 patients
# 9283903 1669135814.2266994
# 9290211 1669136072.7176366
# 9214480 1669136288.9149096
# 9000798 1669136535.808722
# 9246995 1669136724.7937288
# 9068305 1669136923.0118473
# 9316405 1669137114.0410602
# 9075815 1669137294.865748
# 9155449 1669137522.856553
# 9128011 1669137731.843179

# Cluster 10, threads 1, 3 patients
# Start Time  1669137785.699584
# 9283903 1669137785.7000153
# 9171580 1669138188.801546
# 9000798 1669138537.1601562
# 9291234 1669138832.0037203
# 9316405 1669139140.6906438
# 9036316 1669139478.2585301
# 9128011 1669139779.3209581
# 9054866 1669140040.7423177

In [40]:
processed[4]

<Future: pending, key: my_segment_method-38fef8e0535febaebdce3f4820c235bc>

2022-11-23 23:34:19,594 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
2022-11-23 23:36:32,289 - distributed.deploy.cluster - WARNING - Failed to sync cluster info multiple times - perhaps there's a connection issue? Error:
Traceback (most recent call last):
  File "/home/pranjal.sahu/anaconda3/envs/ethicon/lib/python3.8/site-packages/distributed/comm/core.py", line 289, in connect
    comm = await asyncio.wait_for(
  File "/home/pranjal.sahu/anaconda3/envs/ethicon/lib/python3.8/asyncio/tasks.py", line 501, in wait_for
    raise exceptions.TimeoutError()
asyncio.exceptions.TimeoutError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/pranjal.sahu/anaconda3/envs/ethicon/lib/python3.8/site-packages/distributed/deploy/cluster.py", line 131, in _sync_cluster_info
    await self.scheduler_comm.set_metadata(
  File "/home/pranjal.sahu/anaconda3/envs/ethicon/lib/python3.

In [ ]:
# nthreads = 1
# Start Time  1669133566.1740746
# 9283903 1669133566.1742983
# 9290211 1669133892.7999191
# 9214480 1669134189.9747303
# 9000798 1669134565.6692193

In [ ]:
# 6 GPU
# Processing  9075815 1669078836.5805154
# Processing  9036316 1669079092.7211592
# Processing  9155449 1669079313.89807
# Processing  9247140 1669079525.584509
# Processing  9128011 1669079718.9808068
# Processing  9297051 1669079913.9004188
# Processing  9285212 1669080132.2218394
# Processing  9054866 1669080253.5443485
# Processing  9007827 1669080447.7072747
# Processing  9261513 1669080637.7160802
# Processing  9263504 1669080893.8312898
# Processing  9037823 1669080893.9738593
# Processing  9021791 1669081168.2784274
# Processing  9210505 1669081429.1155179
# Processing  9239017 1669081664.124029
# Processing  9268652 1669081936.3881652
# Processing  9135902 1669082137.6828904
# Processing  9252629 1669082362.6144655
# Processing  9028786 1669082549.9535484
# Processing  9277154 1669082802.4558916
# Processing  9103365 1669083017.7320082
# Processing  9016304 1669083243.5094345
# Processing  9233675 1669083459.2613602
# Processing  9235073 1669083664.2123644
# Processing  9212530 1669083882.9608998
# Processing  9040390 1669084121.0219474

In [14]:
# Close the cluster after computation

cluster.close()

In [55]:
len(patients_array)

40

In [11]:
(1667456723.1741402 - 1667451502.621806)/40

130.51380835771562

2022-11-03 02:26:47,242 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


In [10]:
# Main execution loop
# Change the patients_array and months_array based on requirements.
# Following code will run the computation on all samples.

import glob


atlas_path = "s3://oaisample1/ZARRDATA/atlas_image.zarr/scale0"
write_path = "/data/OAIFULLDATA/RESULTS2/"

patients_array = [
    '9283903', '9094865', '9290211', '9171580', '9214480', '9208055',
    '9000798', '9138965', '9246995', '9291234', '9068305', '9101854',
    '9316405', '9305065', '9075815', '9036316', '9155449', '9247140',
    '9128011', '9297051', '9285212', '9054866', '9007827', '9261513',
    '9263504', '9037823', '9021791', '9210505', '9239017', '9268652', 
    '9135902', '9252629', '9028786', '9277154', '9103365', 
    '9016304', '9233675', '9235073', '9212530', '9040390'
]

months_array = [
    'Month-0', 'Month-12', 'Month-24', 'Month-36', 'Month-48', 'Month-72',
    'Month-96'
]

counter = 0

#print(time.time())
#random.shuffle(patients_array)

for patient_id in patients_array:
    import time
    #print('Processing ', patient_id, time.time())
    patient_result = []
    segmentation_result = []
    
    for month in months_array:
#         image_path = "s3://oaisample1/ZARRDATA/PatientID-" + patient_id + '/' + month + '/Images/SAG_3D_DESS_0.zarr/scale0'
        
#         fc_probmap, tc_probmap = dp.segment_method(image_path)
#         phi_AB, image_A, image_B = dp.register_images_delayed(image_path, atlas_path)

#         deformed_fc = dp.deform_probmap_delayed(phi_AB, image_A, image_B,
#                                              fc_probmap, 'FC')
#         deformed_tc = dp.deform_probmap_delayed(phi_AB, image_A, image_B,
#                                              tc_probmap, 'TC')
#         thickness_fc = dp.get_thickness(deformed_fc, 'FC')
#         thickness_tc = dp.get_thickness(deformed_tc, 'TC')
        patient_result.append(0)
        #patient_result.append([deformed_fc, deformed_tc, thickness_fc, thickness_tc])
    
    fc_meshes = []
    tc_meshes = []
    fc_masks = []
    tc_masks = []
    
    #output_result = compute(*patient_result)
    print('----------------------------')
    print(patient_id)
    #for idx, month in enumerate(months_array):
    tc_masks = sorted(glob.glob('/data/OAIFULLDATA/RESULTS2/'+patient_id+'*TC.nii.gz'))
    fc_masks = sorted(glob.glob('/data/OAIFULLDATA/RESULTS2/'+patient_id+'*FC.nii.gz'))
    fc_meshes = sorted(glob.glob('/data/OAIFULLDATA/RESULTS2/'+patient_id+'*inner_FC.vtk'))
    tc_meshes = sorted(glob.glob('/data/OAIFULLDATA/RESULTS2/'+patient_id+'*inner_TC.vtk'))

    for ik, m in enumerate(fc_masks):
        print(' >> ', months_array[ik], m)
        #print(patient_id, counter, months_array[idx], str(counter) + '_itk_deformed_FC.nii.gz')
        #print(patient_id, counter, months_array[idx], str(counter) + '_itk_distance_inner_FC.vtk', str(counter) + '_itk_distance_inner_TC.vtk')
#         itk.imwrite(
#             result[0], write_path + patient_id + '_' +
#             str(counter) + '_itk_deformed_FC.nii.gz')
#         itk.imwrite(
#             result[1], write_path + patient_id + '_' +
#             str(counter) + '_itk_deformed_TC.nii.gz')
        
#         itk.meshwrite(
#             result[2], write_path + patient_id + '_' +
#             str(counter) + '_itk_distance_inner_FC.vtk')
#         itk.meshwrite(
#             result[3], write_path + patient_id + '_' +
#             str(counter) + '_itk_distance_inner_TC.vtk')
        
        counter = counter + 1
    
print(time.time())

----------------------------
9283903
 >>  Month-0 /data/OAIFULLDATA/RESULTS2/9283903_196_itk_deformed_FC.nii.gz
 >>  Month-12 /data/OAIFULLDATA/RESULTS2/9283903_197_itk_deformed_FC.nii.gz
 >>  Month-24 /data/OAIFULLDATA/RESULTS2/9283903_198_itk_deformed_FC.nii.gz
 >>  Month-36 /data/OAIFULLDATA/RESULTS2/9283903_199_itk_deformed_FC.nii.gz
 >>  Month-48 /data/OAIFULLDATA/RESULTS2/9283903_200_itk_deformed_FC.nii.gz
 >>  Month-72 /data/OAIFULLDATA/RESULTS2/9283903_201_itk_deformed_FC.nii.gz
 >>  Month-96 /data/OAIFULLDATA/RESULTS2/9283903_202_itk_deformed_FC.nii.gz
----------------------------
9094865
 >>  Month-0 /data/OAIFULLDATA/RESULTS2/9094865_28_itk_deformed_FC.nii.gz
 >>  Month-12 /data/OAIFULLDATA/RESULTS2/9094865_29_itk_deformed_FC.nii.gz
 >>  Month-24 /data/OAIFULLDATA/RESULTS2/9094865_30_itk_deformed_FC.nii.gz
 >>  Month-36 /data/OAIFULLDATA/RESULTS2/9094865_31_itk_deformed_FC.nii.gz
 >>  Month-48 /data/OAIFULLDATA/RESULTS2/9094865_32_itk_deformed_FC.nii.gz
 >>  Month-72 /data/O

 >>  Month-0 /data/OAIFULLDATA/RESULTS2/9016304_10_itk_deformed_FC.nii.gz
 >>  Month-12 /data/OAIFULLDATA/RESULTS2/9016304_11_itk_deformed_FC.nii.gz
 >>  Month-24 /data/OAIFULLDATA/RESULTS2/9016304_12_itk_deformed_FC.nii.gz
 >>  Month-36 /data/OAIFULLDATA/RESULTS2/9016304_13_itk_deformed_FC.nii.gz
 >>  Month-48 /data/OAIFULLDATA/RESULTS2/9016304_7_itk_deformed_FC.nii.gz
 >>  Month-72 /data/OAIFULLDATA/RESULTS2/9016304_8_itk_deformed_FC.nii.gz
 >>  Month-96 /data/OAIFULLDATA/RESULTS2/9016304_9_itk_deformed_FC.nii.gz
----------------------------
9233675
 >>  Month-0 /data/OAIFULLDATA/RESULTS2/9233675_154_itk_deformed_FC.nii.gz
 >>  Month-12 /data/OAIFULLDATA/RESULTS2/9233675_155_itk_deformed_FC.nii.gz
 >>  Month-24 /data/OAIFULLDATA/RESULTS2/9233675_156_itk_deformed_FC.nii.gz
 >>  Month-36 /data/OAIFULLDATA/RESULTS2/9233675_157_itk_deformed_FC.nii.gz
 >>  Month-48 /data/OAIFULLDATA/RESULTS2/9233675_158_itk_deformed_FC.nii.gz
 >>  Month-72 /data/OAIFULLDATA/RESULTS2/9233675_159_itk_deform

In [ ]:
coiled.create_software_environment(
    name="oaianalysispynvml",
    container="pranjalsahu/oaiimageanalysis",
    pip=["pynvml"],
)


In [3]:
# 1st Step
import coiled

coiled.create_software_environment(
    name="coiledpytorch",
    container="pytorch/pytorch:1.11.0-cuda11.3-cudnn8-runtime",
    conda={
        "channels": [
            "rapidsai",
            "conda-forge",
            "defaults",
        ],
        "dependencies": [
            "python==3.8",
            "dask",
            "pytorch-gpu",
            "coiled-runtime"
        ],
    },
)


Creating new docker_hub build
Creating new software environment
STEP 1: FROM pytorch/pytorch:1.11.0-cuda11.3-cudnn8-runtime
STEP 2: COPY environment.yml environment.yml
--> d5c95491603
STEP 3: RUN conda env update -n coiled -f environment.yml     && rm environment.yml     && conda clean --all -y     && echo "conda activate coiled" >> ~/.bashrc
Solving environment: ...working... done

tbb-2021.7.0         | 2.0 MB    | ########## | 100% 
libcurl-7.86.0       | 348 KB    | ########## | 100% 
libnghttp2-1.47.0    | 808 KB    | ########## | 100% 
_libgcc_mutex-0.1    | 3 KB      | ########## | 100% 
libthrift-0.16.0     | 4.6 MB    | ########## | 100% 
libllvm11-11.1.0     | 28.8 MB   | ########## | 100% 
magma-2.5.4          | 106.8 MB  | ########## | 100% 
s2n-1.0.10           | 442 KB    | ########## | 100% 
pyopenssl-22.1.0     | 122 KB    | ########## | 100% 
aws-c-cal-0.5.11     | 37 KB     | ########## | 100% 
ipython-8.6.0        | 559 KB    | ########## | 100% 
libgcc-ng-12.2.0   

cffi-1.14.4          | 226 KB    | ########## | 100% 
wrapt-1.14.1         | 51 KB     | ########## | 100% 
prompt-toolkit-3.0.3 | 259 KB    | ########## | 100% 
gflags-2.2.2         | 114 KB    | ########## | 100% 
numba-0.55.1         | 3.8 MB    | ########## | 100% 
py-xgboost-1.6.1     | 157 KB    | ########## | 100% 
xz-5.2.6             | 409 KB    | ########## | 100% 
libpng-1.6.38        | 371 KB    | ########## | 100% 
orc-1.7.5            | 1.1 MB    | ########## | 100% 
c-ares-1.18.1        | 113 KB    | ########## | 100% 
backcall-0.2.0       | 13 KB     | ########## | 100% 
heapdict-1.0.1       | 7 KB      | ########## | 100% 
argon2-cffi-bindings | 35 KB     | ########## | 100% 
ptyprocess-0.7.0     | 16 KB     | ########## | 100% 
backports.functools_ | 9 KB      | ########## | 100% 
fsspec-2022.5.0      | 96 KB     | ########## | 100% 
stack_data-0.6.1     | 24 KB     | ########## | 100% 
libgfortran-ng-12.2. | 22 KB     | ########## | 100% 
dask-2022.6.0        | 5 KB 

pyjwt-2.6.0-pyhd8ed1ab_0.tar.bz2              21 KB
beautifulsoup4-4.11.1-pyha770c72_0.tar.bz2      96 KB
idna-3.4-pyhd8ed1ab_0.tar.bz2                 55 KB
bokeh-2.4.3-pyhd8ed1ab_3.tar.bz2            13.3 MB
setuptools-65.5.1-pyhd8ed1ab_0.tar.bz2       731 KB
traitlets-5.5.0-pyhd8ed1ab_0.tar.bz2          85 KB
json5-0.9.5-pyh9f0ad1d_0.tar.bz2              20 KB
libcblas-3.9.0-16_linux64_mkl.tar.bz2         12 KB
python-fastjsonschema-2.16.2-pyhd8ed1ab_0.tar.bz2     242 KB
libedit-3.1.20191231-he28a2e2_2.tar.bz2      121 KB
python-3.8.0-h357f687_5.tar.bz2             38.4 MB
packaging-21.3-pyhd8ed1ab_0.tar.bz2           36 KB
pandas-1.4.2-py38h47df419_2.tar.bz2         12.6 MB
pytz-2022.6-pyhd8ed1ab_0.tar.bz2             235 KB
tornado-6.1-py38h0a891b7_3.tar.bz2           645 KB
webencodings-0.5.1-py_1.tar.bz2               12 KB
readline-8.1.2-h0f457ee_0.tar.bz2            291 KB
google-auth-oauthlib-0.7.1-pyhd8ed1ab_0.tar.bz2      21 KB
cloudpickle-2.1.0-pyhd8ed1ab_0.tar.bz2        

jedi-0.18.1-pyhd8ed1ab_2.tar.bz2             799 KB
_py-xgboost-mutex-2.0-cpu_0.tar.bz2            8 KB
google-cloud-core-2.3.2-pyhd8ed1ab_0.tar.bz2      28 KB
future-0.18.2-pyhd8ed1ab_6.tar.bz2           356 KB
coiled-runtime-0.1.1-hd8ed1ab_0.tar.bz2        5 KB
pyarrow-8.0.0-py38h9f6a473_5_cpu.tar.bz2     3.3 MB
dataclasses-0.8-pyhc8e2a94_3.tar.bz2          10 KB
pyzmq-24.0.1-py38hfc09fa9_1.tar.bz2          482 KB
aws-c-common-0.6.2-h7f98852_0.tar.bz2        168 KB
jinja2-3.1.2-pyhd8ed1ab_1.tar.bz2             99 KB
notebook-shim-0.2.2-pyhd8ed1ab_0.tar.bz2      15 KB
krb5-1.19.3-h3790be6_0.tar.bz2               1.4 MB
nccl-2.14.3.1-h0800d71_0.tar.bz2           145.2 MB
entrypoints-0.4-pyhd8ed1ab_0.tar.bz2           9 KB
grpcio-1.46.3-py38ha0cdfde_0.tar.bz2         2.6 MB
dask-core-2022.6.0-pyhd8ed1ab_0.tar.bz2      834 KB
typing-extensions-4.4.0-hd8ed1ab_0.tar.bz2       8 KB
sqlite-3.40.0-h4ff8645_0.tar.bz2             801 KB
toolz-0.11.2-pyhd8ed1ab_0.tar.bz2             48 KB
decora

Removed distributed-2022.6.0-pyhd8ed1ab_0.tar.bz2
Removed packaging-21.3-pyhd8ed1ab_0.tar.bz2
removing xgboost-1.6.1-cpu_py38h66f0ec1_1
Removed asciitree-0.3.3-py_2.tar.bz2
Removed libev-4.33-h516909a_1.tar.bz2
removing nbconvert-pandoc-7.2.5-pyhd8ed1ab_0
_py-xgboost-mutex-2.0-cpu_0                   22 KB
Removed libedit-3.1.20191231-he28a2e2_2.tar.bz2
Removed numcodecs-0.10.2-py38hfa26641_0.tar.bz2
removing dask-2022.6.0-pyhd8ed1ab_0
removing nbconvert-7.2.5-pyhd8ed1ab_0
removing typing-extensions-4.4.0-hd8ed1ab_0
removing _libgcc_mutex-0.1-conda_forge
removing pytorch-gpu-1.11.0-cuda112py38h68407e5_1
removing liblapack-3.9.0-16_linux64_mkl

removing coiled-runtime-0.1.1-hd8ed1ab_0

dask-2022.6.0-pyhd8ed1ab_0                    14 KB
python_abi-3.8-2_cp38                         11 KB
removing libcblas-3.9.0-16_linux64_mkl
libgfortran-ng-12.2.0-h69a702a_19             97 KB
pytorch-gpu-1.11.0-cuda112py38h68407e5_1      27 KB
xgboost-1.6.1-cpu_py38h66f0ec1_1              40 KB
liblapa

ServerError: Internal error occured, contact customer service and quote 3b17ad37-a078-48ea-bf29-903937191f37